In [ ]:
"""mc_circle_qrng.ipynb"""
# Cell 1

from __future__ import annotations

import typing

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.markers import MarkerStyle
from matplotlib.patches import Circle
from numba import float64, vectorize  # type: ignore
from numpy.random import default_rng

if typing.TYPE_CHECKING:
    from matplotlib.axes import Axes
    from numpy.random import Generator
    from numpy.typing import NDArray

%matplotlib widget


def plot_prng(ax: Axes) -> None:
    iterations_sqrt: int = 320
    iterations: int = iterations_sqrt**2

    rng: Generator = default_rng(seed=2020)
    x: NDArray[np.float_] = rng.random(iterations) * 2 - 1
    y: NDArray[np.float_] = rng.random(iterations) * 2 - 1

    d: NDArray[np.float_] = x**2 + y**2

    x_in: NDArray[np.float_] = x[d <= 1.0]
    y_in: NDArray[np.float_] = y[d <= 1.0]
    x_out: NDArray[np.float_] = x[d > 1.0]
    y_out: NDArray[np.float_] = y[d > 1.0]

    pixel_size: float = (72 / ax.figure.dpi) ** 2  # type: ignore
    ax.scatter(x_in, y_in, color="red", marker=MarkerStyle("."), s=pixel_size)
    ax.scatter(x_out, y_out, color="blue", marker=MarkerStyle("."), s=pixel_size)

    ax.add_patch(Circle((0, 0), radius=1.0, color="green", fill=False))

    est_area: float = np.count_nonzero(d <= 1.0) / iterations * 4
    act_area: float = np.pi
    err: float = (est_area - act_area) / act_area

    ax.set_title("Circle Area via Monte Carlo Estimation (Numpy PRNG)")
    ax.set_xlim(-1.1, 1.1)
    ax.set_ylim(-1.1, 1.1)
    ax.axhline(0, color="gray")
    ax.axvline(0, color="gray")
    ax.set_aspect("equal")

    # fmt: off
    ax.text(-0.50, -1.5,
             "Total dots\nAct. Area\nPRNG Est. Area\nPRNG % Rel Err", ha="left")

    ax.text(0.10, -1.5,
             f"= {iterations:,}\n= {act_area:.6f}\n= {est_area:.6f}\n= {err:.6%}",
             ha="left")
    # fmt: on


@vectorize([float64(float64, float64)], nopython=True)  # type: ignore
def halton(n: float, p: int) -> float:
    h = 0
    f = 1
    while n > 0:
        f: float = f / p
        h += (n % p) * f
        n = int(n / p)
    return h


def plot_qrng(ax: Axes) -> None:
    iterations_sqrt: int = 160
    iterations: int = iterations_sqrt**2

    primes: list[int] = [2, 3]

    x: NDArray[np.float_] = halton(np.arange(iterations), primes[0]) * 2 - 1  # type: ignore
    y: NDArray[np.float_] = halton(np.arange(iterations), primes[1]) * 2 - 1  # type: ignore

    d: NDArray[np.float_] = x**2 + y**2

    x_in: NDArray[np.float_] = x[d <= 1.0]
    y_in: NDArray[np.float_] = y[d <= 1.0]

    x_out: NDArray[np.float_] = x[d > 1.0]
    y_out: NDArray[np.float_] = y[d > 1.0]

    pixel_size: float = (72 / ax.figure.dpi) ** 2  # type: ignore
    ax.scatter(x_in, y_in, color="red", marker=MarkerStyle("."), s=pixel_size)
    ax.scatter(x_out, y_out, color="blue", marker=MarkerStyle("."), s=pixel_size)

    ax.add_patch(Circle((0, 0), radius=1.0, color="green", fill=False))

    est_area: float = np.count_nonzero(d <= 1.0) / iterations * 4
    act_area: float = np.pi
    err: float = (est_area - act_area) / act_area

    ax.set_title("Circle Area via Monte Carlo Estimation (Halton QRNG)")
    ax.set_xlim(-1.1, 1.1)
    ax.set_ylim(-1.1, 1.1)
    ax.axhline(0, color="gray")
    ax.axvline(0, color="gray")
    ax.set_aspect("equal")

    # fmt: off
    ax.text(-0.50, -1.5,
             "Total dots\nAct. Area\nQRNG Est. Area\nQRNG % Rel Err", ha="left")

    ax.text(0.10, -1.5,
             f"= {iterations:,}\n= {act_area:.6f}\n= {est_area:.6f}\n= {err:.6%}",
             ha="left")
    # fmt: on


def main() -> None:
    plt.close("all")
    plt.figure(" ", figsize=(11, 6), constrained_layout=True)
    plot_prng(plt.subplot(121))
    plot_qrng(plt.subplot(122))
    plt.show()


main()